In [ ]:
# Dependencies
import pandas as pd
import datetime as dt
import numpy as np
import glob
import os
  
# Downloaded csv files saved outside respository due to size limits
# Please view readme for file locations
# Merging the files
joined_files = os.path.join("", "20220*.csv")a
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [ ]:
# Initial set up and cleaning
df.dropna(how='any')
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['trip_duration'] = df['ended_at'] - df['started_at']
df['month'] = pd.to_datetime(df['started_at']).dt.to_period('M')
df['day_of_week'] = df['started_at'].dt.day_name()
df = df.loc[df['trip_duration'] < '01:00:00', :]
df = df[df['rideable_type'] != 'docked_bike']
df['start_lat'] = df['start_lat'].round(3)
df['start_lng'] = df['start_lng'].round(3)
df['end_lat'] = df['end_lat'].round(3)
df['end_lng'] = df['end_lng'].round(3)

In [ ]:
# Creating time bins for later grouping
bins = [pd.Timedelta(minutes = 0),
        pd.Timedelta(minutes = 10),
        pd.Timedelta(minutes = 20),
        pd.Timedelta(minutes = 30),
        pd.Timedelta(minutes = 40),
        pd.Timedelta(minutes = 50),
        pd.Timedelta(hours = 4)]
labels = ['0-10min', '11-20min', '21-30min', '31-40min', '41-50min', '51min+']
df['time_bin'] = pd.cut(df['trip_duration'], bins, labels = labels)

In [ ]:
# Shape of dataframe to better grasp size
df.shape

In [ ]:
# Copying df to maintain a clean base
df2 = df.copy()
df2 = df2.dropna(how='any')

In [ ]:
# Station summary data set up
# Grouping by month/name
df_sstation = df.groupby([df2['month'], df2['start_station_name'].rename('station')])
df_estation = df.groupby([df2['month'], df2['end_station_name'].rename('station')])
# Counting number of start and end stations
sstation = df_sstation['start_station_name'].count().rename('scount')
estation = df_estation['end_station_name'].count().rename('ecount')
# Df for counts
df_start = pd.DataFrame(sstation)
df_end = pd.DataFrame(estation)

In [ ]:
# Merging two df together
station_summary = pd.merge(df_start, df_end, how='outer', on=['month', 'station'])
# Creating the df and making na values 0
df_station_summary = pd.DataFrame(station_summary)
df_station_summary = df_station_summary.fillna(0)
# Total counts combined end and start
df_station_summary['total_s_e_counts'] = df_station_summary['scount'] + df_station_summary['ecount']

In [ ]:
# Creating CSV for Tableau
df_station_summary.to_csv('Resources/station_summary.csv')

In [ ]:
# Lat/Lng table for starting location
df_slocation = df2.filter(['start_station_name','start_lat','start_lng'], axis = 1).copy()
df_slocation = df_slocation.sort_values('start_station_name')
df_slocation = df_slocation.dropna(how='any')
df_slocation_unique = df_slocation.drop_duplicates(subset = 'start_station_name', keep = 'first')

In [ ]:
# Creating CSV for Tableau
df_slocation_unique.to_csv('Resources/slocation_table.csv')

In [ ]:
# Lat/Lng table for ending location
df_elocation = df2.filter(['end_station_name','end_lat','end_lng'], axis = 1).copy()
df_elocation = df_elocation.sort_values('end_station_name')
df_elocation = df_elocation.dropna(how='any')
df_elocation_unique = df_elocation.drop_duplicates(subset = 'end_station_name', keep = 'first')

In [ ]:
# Creating CSV for Tableau
df_elocation_unique.to_csv('Resources/elocation_table.csv')

In [ ]:
# Member/Casual preference on type of bike
df_type = df2.groupby([df2['member_casual']])
classic_total = df_type['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
electric_total = df_type['rideable_type'].apply(lambda x: (x=='electric_bike').sum())

In [ ]:
# Creating df 
type_table = pd.DataFrame({
    'Classic': classic_total,
    'Electric': electric_total
})

In [ ]:
# Creating CSV for Tableau
type_table.to_csv('Resources/type_table.csv')

In [ ]:
# Average duration by bike type
df_type_duration = df2.groupby([df2['rideable_type']])
type_duration = df_type_duration['trip_duration'].mean()

In [ ]:
# Creating CSV for Tableau
type_duration.to_csv('Resources/type_duration.csv')

In [ ]:
# Total counts by station and hour to see busiest times by station in a given month
df_hour = df2.groupby([df2['month'], df2['started_at'].dt.hour.rename('hour')])
hour_total = df_hour['started_at'].count().rename('counts')

In [ ]:
# Creating CSV for Tableau
hour_total.to_csv('Resources/hour_total.csv')

In [ ]:
# Total counts by DOW and average trip times
df_dayofweek = df2.groupby([df2['month'], df2['started_at'].dt.hour.rename('hour'), df2['day_of_week']])
dow_total = df_dayofweek['day_of_week'].count().rename('counts')
dow_duration = df_dayofweek['trip_duration'].mean()

In [ ]:
# Creating df
dow_duration_table = pd.DataFrame({
    'dow_total': dow_total,
    'dow_duration': dow_duration,
})

In [ ]:
# Creating CSV for Tableau
dow_duration_table.to_csv('Resources/dow_duration_table.csv')

In [ ]:
# Total counts by station and DOW
df_station_dayofweek = df2.groupby([df2['month'], df2['start_station_name'], df2['day_of_week']])
dow_station_total = df_station_dayofweek['day_of_week'].count().rename('counts')

In [ ]:
# Creating CSV for Tableau
dow_station_total.to_csv('Resources/dow_station_total.csv')

In [ ]:
# Total counts by station and time bins
df_bin_count = df2.groupby([df2['month'], df2['start_station_name'], df2['time_bin']])
dow_bin_total = df_bin_count['started_at'].count().rename('counts')

In [ ]:
# Creating CSV for Tableau
dow_bin_total.to_csv('Resources/dow_bin_total.csv')

In [ ]:
# Creating summary table for high level values
df_summary = df2.groupby([df2['month']])
ride_total = df_summary['ride_id'].count()
members_total = df_summary['member_casual'].apply(lambda x: (x=='member').sum())
casual_total = df_summary['member_casual'].apply(lambda x: (x=='casual').sum())
classic_total = df_summary['rideable_type'].apply(lambda x: (x=='classic_bike').sum())
electric_total = df_summary['rideable_type'].apply(lambda x: (x=='electric_bike').sum())
duration_avg = df_summary['trip_duration'].mean()

In [ ]:
# Creating df
summary_table = pd.DataFrame({
    'total_rides': ride_total,
    'total_member': members_total,
    'total_casual': casual_total,
    'total_classic': classic_total,
    'total_electric': electric_total,
    'avg_duration': duration_avg
})

In [ ]:
# Creating CSV for Tableau
summary_table.to_csv('Resources/summary_table.csv')